# SQL query from table names - Continued

In [20]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [21]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [22]:
context = [ {'role':'system', 'content':"""
You are a SQL expert. I will provide you with table definitions and sample data, and you will help me write SQL queries.

CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name VARCHAR(100)
);

INSERT INTO employees VALUES
(1, 'John Smith'),
(2, 'Jane Doe'),
(3, 'Bob Wilson');

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

INSERT INTO salary VALUES
(1, '2023-01-01', 50000),
(2, '2023-01-01', 60000),
(3, '2023-01-01', 55000);

CREATE TABLE studies (
    ID INT PRIMARY KEY,
    ID_usr INT,
    educational_level INT,
    Institution VARCHAR(100),
    Years DATE,
    Speciality VARCHAR(100),
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

INSERT INTO studies VALUES
(1, 1, 3, 'MIT', '2020-01-01', 'Computer Science'),
(2, 2, 4, 'Stanford', '2019-01-01', 'Data Science'),
(3, 3, 3, 'Harvard', '2021-01-01', 'Business');
"""} ]

In [23]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
 -- Example queries:
 
 -- Q: Show me all employees with their salaries
SELECT e.name, s.salary 
FROM employees e 
JOIN salary s ON e.ID_usr = s.ID_usr;

-- Q: Find employees with highest education level
SELECT e.name, s.educational_level, s.Institution 
FROM employees e 
JOIN studies s ON e.ID_usr = s.ID_usr 
WHERE s.educational_level = (SELECT MAX(educational_level) FROM studies);

-- Q: Get average salary by education level
SELECT s2.educational_level, AVG(s1.salary) as avg_salary 
FROM salary s1 
JOIN studies s2 ON s1.ID_usr = s2.ID_usr 
GROUP BY s2.educational_level;
"""
})

In [24]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [25]:
# Probar el nuevo prompt
context_user = context.copy()
test_queries = [
    "List all employees and their specialities",
    "Show me the average salary by institution",
    "Find employees with salary above average"
]

for query in test_queries:
    print(f"\nQuestion: {query}")
    print(return_CCRMSQL(query, context_user))


Question: List all employees and their specialities
```sql
SELECT e.name, st.Speciality
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr;
```

Question: Show me the average salary by institution
```sql
SELECT st.Institution, AVG(sa.salary) as avg_salary
FROM salary sa
JOIN studies st ON sa.ID_usr = st.ID_usr
GROUP BY st.Institution;
```

Question: Find employees with salary above average
```sql
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > (SELECT AVG(salary) FROM salary);
```


In [26]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("List all employees and their specialities", old_context_user))

This is your SQL:
```sql
SELECT employees.name, studies.speciality
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr;
```

This SQL query retrieves the names of employees along with their specialities by performing an inner join between the "employees" table and the "studies" table on the common column "ID_usr".


In [27]:
#new
print(return_CCRMSQL("List all employees and their specialities", context_user))

```sql
SELECT e.name, st.Speciality
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr;
```


In [28]:
#old
print(return_CCRMSQL("List all employees and their specialities", old_context_user))

This is your SQL:
```sql
SELECT employees.name, studies.speciality
FROM employees
JOIN studies ON employees.ID_usr = studies.ID_usr;
```

This SQL query retrieves the names of employees along with their specialities by performing an inner join between the "employees" and "studies" tables on the common column "ID_usr".


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [29]:
context_v1 = [ {'role':'system', 'content':"""
You are a SQL expert. I will provide you with table schemas and relationships, and you will help me write SQL queries.

-- Database Schema with Relationships:
CREATE TABLE employees (
    ID_usr INT PRIMARY KEY,
    name VARCHAR(100),
    hire_date DATE
);

CREATE TABLE salary (
    ID_usr INT,
    year DATE,
    salary FLOAT,
    bonus FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

CREATE TABLE studies (
    ID INT PRIMARY KEY,
    ID_usr INT,
    educational_level INT, -- 1:High School, 2:Bachelor, 3:Master, 4:PhD
    Institution VARCHAR(100),
    Years DATE,
    Speciality VARCHAR(100),
    GPA FLOAT,
    FOREIGN KEY (ID_usr) REFERENCES employees(ID_usr)
);

-- Sample Data
INSERT INTO employees VALUES
(1, 'John Smith', '2020-01-15'),
(2, 'Jane Doe', '2019-03-20'),
(3, 'Bob Wilson', '2021-06-10');

INSERT INTO salary VALUES
(1, '2023-01-01', 50000, 5000),
(2, '2023-01-01', 60000, 6000),
(3, '2023-01-01', 55000, 4000);

INSERT INTO studies VALUES
(1, 1, 3, 'MIT', '2020-01-01', 'Computer Science', 3.8),
(2, 2, 4, 'Stanford', '2019-01-01', 'Data Science', 4.0),
(3, 3, 3, 'Harvard', '2021-01-01', 'Business', 3.9);

-- Example Queries with Explanations:
-- 1. Complex join with aggregation
SELECT e.name, 
       s2.Institution,
       s1.salary + s1.bonus as total_compensation
FROM employees e
JOIN salary s1 ON e.ID_usr = s1.ID_usr
JOIN studies s2 ON e.ID_usr = s2.ID_usr
WHERE s2.GPA > 3.5
ORDER BY total_compensation DESC;

-- 2. Subquery with aggregation
SELECT e.name,
       s.educational_level,
       s.Institution
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.GPA > (SELECT AVG(GPA) FROM studies);
"""} ]

In [30]:
context_v2 = [ {'role':'system', 'content':"""
You are a SQL expert helping HR analysts. Focus on business insights from employee data.

-- Tables Overview:
employees: Contains basic employee information
salary: Tracks compensation history
studies: Educational background and qualifications

-- Schema and Relationships:
[Previous CREATE TABLE statements...]

-- Business-Oriented Example Queries:
-- 1. Compensation Analysis
SELECT 
    s2.educational_level,
    AVG(s1.salary) as avg_salary,
    COUNT(DISTINCT e.ID_usr) as employee_count
FROM employees e
JOIN salary s1 ON e.ID_usr = s1.ID_usr
JOIN studies s2 ON e.ID_usr = s2.ID_usr
GROUP BY s2.educational_level
ORDER BY avg_salary DESC;

-- 2. Education ROI Analysis
SELECT 
    s.Institution,
    AVG(sal.salary) as avg_salary,
    COUNT(*) as graduate_count
FROM studies s
JOIN salary sal ON s.ID_usr = sal.ID_usr
GROUP BY s.Institution
HAVING COUNT(*) > 1;
"""} ]

In [31]:
context_v3 = [ {'role':'system', 'content':"""
You are a SQL expert. Here are common query patterns for our employee database:

Pattern 1 - Basic Joins:
SELECT e.*, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr;

Pattern 2 - Aggregations with GROUP BY:
SELECT s.educational_level, 
       COUNT(*) as count,
       AVG(sal.salary) as avg_salary
FROM studies s
JOIN salary sal ON s.ID_usr = sal.ID_usr
GROUP BY s.educational_level;

Pattern 3 - Subqueries:
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.salary > (SELECT AVG(salary) FROM salary);

Pattern 4 - Multiple Joins with Conditions:
SELECT e.name, s1.salary, s2.Institution
FROM employees e
JOIN salary s1 ON e.ID_usr = s1.ID_usr
JOIN studies s2 ON e.ID_usr = s2.ID_usr
WHERE s2.educational_level >= 3;
"""} ]

In [32]:
test_queries = [
    "Show me the average salary by education level and institution",
    "Find employees with highest salary in their education level",
    "Calculate the salary difference between PhD and Master's degree holders"
]

# Probar cada versión
versions = [
    ("Version 1 - Detailed", context_v1),
    ("Version 2 - Business", context_v2),
    ("Version 3 - Patterns", context_v3)
]

for version_name, context in versions:
    print(f"\n=== Testing {version_name} ===")
    for query in test_queries:
        print(f"\nQuestion: {query}")
        print(return_CCRMSQL(query, context))
        print("-" * 50)


=== Testing Version 1 - Detailed ===

Question: Show me the average salary by education level and institution
To show the average salary by education level and institution, you can use the following SQL query:

```sql
SELECT s.educational_level,
       s.Institution,
       AVG(salary + bonus) AS avg_total_compensation
FROM employees e
JOIN salary sal ON e.ID_usr = sal.ID_usr
JOIN studies s ON e.ID_usr = s.ID_usr
GROUP BY s.educational_level, s.Institution;
```

This query joins the `employees`, `salary`, and `studies` tables on the appropriate keys, calculates the average total compensation (salary + bonus) for each education level and institution, and groups the results by educational level and institution.
--------------------------------------------------

Question: Find employees with highest salary in their education level
To find employees with the highest salary in their education level, you can use the following SQL query:

```sql
WITH max_salary_per_education_level AS (
    

# Laboratorio de SQL Query Generation - Informe de Implementación

## Resumen Ejecutivo
Este laboratorio exploró tres enfoques diferentes para mejorar la generación de consultas SQL utilizando LLMs, basándose en las recomendaciones del paper "How to Prompt LLMs for Text-to-SQL" de la Universidad de Ohio. Se implementaron y evaluaron diferentes estrategias de prompting para optimizar la generación de consultas SQL.

## Objetivos del Laboratorio
- Implementar diferentes técnicas de prompting para SQL
- Evaluar la efectividad de distintos enfoques de estructuración
- Comparar resultados entre diferentes versiones
- Identificar las mejores prácticas para generación SQL

## Implementaciones

### 1. Versión Detallada con Relaciones
**Objetivo**: Maximizar la comprensión de la estructura de datos y relaciones.

**Características**:
- Esquema completo de base de datos
- Datos de ejemplo detallados
- Relaciones explícitas entre tablas
- Ejemplos de queries complejas

**Resultados**:
- Alta precisión en joins complejos
- Buen manejo de relaciones
- Queries bien estructuradas

### 2. Versión Orientada a Negocio
**Objetivo**: Enfatizar el contexto de negocio y análisis.

**Características**:
- Enfoque en casos de uso de HR
- Ejemplos de análisis de compensación
- Métricas de negocio relevantes
- Queries orientadas a insights

**Resultados**:
- Mejor comprensión del contexto
- Queries más relevantes al negocio
- Buenos resultados en análisis complejos

### 3. Versión de Patrones
**Objetivo**: Proporcionar templates y patrones comunes.

**Características**:
- Patrones de query establecidos
- Ejemplos de estructuras comunes
- Templates reutilizables
- Casos de uso típicos

**Resultados**:
- Consistencia en estructura
- Queries más predecibles
- Mejor reusabilidad

## Análisis Técnico

### Fortalezas
1. **Precisión**:
   - Mejor comprensión de relaciones
   - Queries sintácticamente correctas
   - Resultados consistentes

2. **Versatilidad**:
   - Adaptabilidad a diferentes casos
   - Manejo de múltiples escenarios
   - Flexibilidad en implementación

3. **Usabilidad**:
   - Queries bien estructuradas
   - Resultados comprensibles
   - Fácil adaptación

### Áreas de Mejora
1. **Complejidad**:
   - Manejo de casos edge
   - Optimización de queries
   - Consistencia en resultados complejos

2. **Limitaciones**:
   - Dependencia del contexto
   - Necesidad de ejemplos específicos
   - Variabilidad en resultados

## Lecciones Aprendidas

### Diseño de Prompts
1. Importancia de estructura clara
2. Valor de ejemplos específicos
3. Necesidad de contexto adecuado

### Generación SQL
1. Beneficios de patrones establecidos
2. Importancia de relaciones claras
3. Valor del contexto de negocio

### Mejores Prácticas
1. Incluir datos de ejemplo
2. Establecer relaciones explícitas
3. Proporcionar ejemplos relevantes

## Recomendaciones

### Mejoras Técnicas
1. Implementar validación de queries
2. Desarrollar biblioteca de patrones
3. Crear sistema de feedback

### Mejoras de Proceso
1. Establecer estándares de prompting
2. Implementar testing sistemático
3. Desarrollar documentación detallada

## Conclusión
El laboratorio demostró la efectividad de diferentes enfoques de prompting para generación SQL, con cada versión mostrando fortalezas en diferentes aspectos. La combinación de estructura clara, contexto de negocio y patrones establecidos produce los mejores resultados.

## Próximos Pasos
1. Expandir biblioteca de patrones
2. Implementar sistema de validación
3. Desarrollar más casos de uso
4. Crear documentación comprensiva